This notebook finds random cells from each prediction category and displays them. The purpose is to get representative images examples of each category.

In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# import pillow and open cv
import PIL
import seaborn as sns
import tifffile as tf
from cytocherrypick import cherrypick
from PIL import Image, ImageEnhance

In [2]:
# parameters
CELL_TYPE = "SHSY5Y"
feature = "Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256"

In [3]:
# define directories
# where the images are
image_dir_path = pathlib.Path(
    "/media/lippincm/18T/interstellar_data/70117_20230210MM1_Gasdermin514_CP_BC430856__2023-03-22T15_42_38-Measurement1/2.IC/"
)
# if path does not exist, create it
image_dir_path.mkdir(parents=True, exist_ok=True)

image_out_dir_path = pathlib.Path("../figures/")
# if path does not exist, create it
image_out_dir_path.mkdir(parents=True, exist_ok=True)

In [4]:
df_path = pathlib.Path(
    f"../../4.sc_Morphology_Neural_Network_MLP_Model/results/Multi_Class/MultiClass_MLP/{CELL_TYPE}/single_cell_predictions.parquet"
)
# read in the data
df = pd.read_parquet(df_path)

# df_no_fs_path = pathlib.Path(f"../../data/{cell_type}_sc.parquet")
# # read in the data
# df_no_fs = pd.read_parquet(df_no_fs_path)

df.head()

,true_label,predicted_label,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,...,Metadata_Dose,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,labels,data_split,shuffle
0,1,1,SH-SY5Y,G21,4237,7,6,Disulfiram,1.000,µM,...,10.000_µg_per_ml,859.550103,1618.409051,919.0,1712.0,767.0,1519.0,pyroptosis,train,True
1,1,1,SH-SY5Y,K19,3548,8,6,Z-VAD-FMK,100.000,µM,...,100.000_µM,592.736056,504.957400,660.0,569.0,473.0,454.0,healthy,train,True
2,1,1,SH-SY5Y,D22,3364,12,6,DMSO,0.025,%,...,100.000_µg_per_ml_10.000_µM,1475.095941,1240.112283,1534.0,1300.0,1392.0,1158.0,pyroptosis,train,True
3,2,1,SH-SY5Y,M15,4394,15,6,DMSO,0.025,%,...,5.000_nM,826.263944,1299.477903,873.0,1423.0,751.0,1249.0,healthy,train,True
4,1,1,SH-SY5Y,C14,4260,2,6,DMSO,0.025,%,...,0.100_%,464.858182,1043.416364,495.0,1168.0,439.0,997.0,healthy,train,True


In [5]:
# add column for if the prediction was correct
df["correct"] = df.apply(lambda x: x["true_label"] == x["predicted_label"], axis=1)
# split the data into correct and incorrect
df_correct = df[df["correct"] == True]
df_incorrect = df[df["correct"] == False]
assert len(df_correct) + len(df_incorrect) == len(df)

In [6]:
df.head()

,true_label,predicted_label,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,...,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,labels,data_split,shuffle,correct
0,1,1,SH-SY5Y,G21,4237,7,6,Disulfiram,1.000,µM,...,859.550103,1618.409051,919.0,1712.0,767.0,1519.0,pyroptosis,train,True,True
1,1,1,SH-SY5Y,K19,3548,8,6,Z-VAD-FMK,100.000,µM,...,592.736056,504.957400,660.0,569.0,473.0,454.0,healthy,train,True,True
2,1,1,SH-SY5Y,D22,3364,12,6,DMSO,0.025,%,...,1475.095941,1240.112283,1534.0,1300.0,1392.0,1158.0,pyroptosis,train,True,True
3,2,1,SH-SY5Y,M15,4394,15,6,DMSO,0.025,%,...,826.263944,1299.477903,873.0,1423.0,751.0,1249.0,healthy,train,True,False
4,1,1,SH-SY5Y,C14,4260,2,6,DMSO,0.025,%,...,464.858182,1043.416364,495.0,1168.0,439.0,997.0,healthy,train,True,True


In [7]:
df["data_split"].unique()
df["shuffle"].unique()

array([ True, False])

In [8]:
# split the data into the different classes
pyroptosis_df = df_correct[df_correct["labels"] == "pyroptosis"]
apoptosis_df = df_correct[df_correct["labels"] == "apoptosis"]
control_df = df_correct[df_correct["labels"] == "healthy"]

# split the data classes by shuffled and unshuffled
pyroptosis_shuffled_df = pyroptosis_df[pyroptosis_df["shuffle"] == True]
pyroptosis_unshuffled_df = pyroptosis_df[pyroptosis_df["shuffle"] == False]
apoptosis_shuffled_df = apoptosis_df[apoptosis_df["shuffle"] == True]
apoptosis_unshuffled_df = apoptosis_df[apoptosis_df["shuffle"] == False]
control_shuffled_df = control_df[control_df["shuffle"] == True]
control_unshuffled_df = control_df[control_df["shuffle"] == False]

# split the shuffled/unshuffled data by the data splits
pyroptosis_shuffled_train_df = pyroptosis_shuffled_df[
    pyroptosis_shuffled_df["data_split"] == "train"
]
pyroptosis_shuffled_test_df = pyroptosis_shuffled_df[
    pyroptosis_shuffled_df["data_split"] == "test"
]
pyroptosis_shuffled_validation_df = pyroptosis_shuffled_df[
    pyroptosis_shuffled_df["data_split"] == "validation"
]
pyroptosis_shuffled_treatment_holdout_df = pyroptosis_shuffled_df[
    pyroptosis_shuffled_df["data_split"] == "treatment_holdout"
]
pyroptosis_shuffled_holdout_df = pyroptosis_shuffled_df[
    pyroptosis_shuffled_df["data_split"] == "holdout"
]

pyroptosis_unshuffled_train_df = pyroptosis_unshuffled_df[
    pyroptosis_unshuffled_df["data_split"] == "train"
]
pyroptosis_unshuffled_test_df = pyroptosis_unshuffled_df[
    pyroptosis_unshuffled_df["data_split"] == "test"
]
pyroptosis_unshuffled_validation_df = pyroptosis_unshuffled_df[
    pyroptosis_unshuffled_df["data_split"] == "validation"
]
pyroptosis_unshuffled_treatment_holdout_df = pyroptosis_unshuffled_df[
    pyroptosis_unshuffled_df["data_split"] == "treatment_holdout"
]
pyroptosis_unshuffled_holdout_df = pyroptosis_unshuffled_df[
    pyroptosis_unshuffled_df["data_split"] == "holdout"
]

apoptosis_shuffled_train_df = apoptosis_shuffled_df[
    apoptosis_shuffled_df["data_split"] == "train"
]
apoptosis_shuffled_test_df = apoptosis_shuffled_df[
    apoptosis_shuffled_df["data_split"] == "test"
]
apoptosis_shuffled_validation_df = apoptosis_shuffled_df[
    apoptosis_shuffled_df["data_split"] == "validation"
]
apoptosis_shuffled_treatment_holdout_df = apoptosis_shuffled_df[
    apoptosis_shuffled_df["data_split"] == "treatment_holdout"
]
apoptosis_shuffled_holdout_df = apoptosis_shuffled_df[
    apoptosis_shuffled_df["data_split"] == "holdout"
]

apoptosis_unshuffled_train_df = apoptosis_unshuffled_df[
    apoptosis_unshuffled_df["data_split"] == "train"
]
apoptosis_unshuffled_test_df = apoptosis_unshuffled_df[
    apoptosis_unshuffled_df["data_split"] == "test"
]
apoptosis_unshuffled_validation_df = apoptosis_unshuffled_df[
    apoptosis_unshuffled_df["data_split"] == "validation"
]
apoptosis_unshuffled_treatment_holdout_df = apoptosis_unshuffled_df[
    apoptosis_unshuffled_df["data_split"] == "treatment_holdout"
]
apoptosis_unshuffled_holdout_df = apoptosis_unshuffled_df[
    apoptosis_unshuffled_df["data_split"] == "holdout"
]

control_shuffled_train_df = control_shuffled_df[
    control_shuffled_df["data_split"] == "train"
]
control_shuffled_test_df = control_shuffled_df[
    control_shuffled_df["data_split"] == "test"
]
control_shuffled_validation_df = control_shuffled_df[
    control_shuffled_df["data_split"] == "validation"
]
control_shuffled_treatment_holdout_df = control_shuffled_df[
    control_shuffled_df["data_split"] == "treatment_holdout"
]
control_shuffled_holdout_df = control_shuffled_df[
    control_shuffled_df["data_split"] == "holdout"
]

control_unshuffled_train_df = control_unshuffled_df[
    control_unshuffled_df["data_split"] == "train"
]
control_unshuffled_test_df = control_unshuffled_df[
    control_unshuffled_df["data_split"] == "test"
]
control_unshuffled_validation_df = control_unshuffled_df[
    control_unshuffled_df["data_split"] == "validation"
]
control_unshuffled_treatment_holdout_df = control_unshuffled_df[
    control_unshuffled_df["data_split"] == "treatment_holdout"
]
control_unshuffled_holdout_df = control_unshuffled_df[
    control_unshuffled_df["data_split"] == "holdout"
]

# add each df to a dictionary
dict_of_dfs = {}
dict_of_dfs["pyroptosis_shuffled_train_df"] = pyroptosis_shuffled_train_df
dict_of_dfs["pyroptosis_shuffled_test_df"] = pyroptosis_shuffled_test_df
dict_of_dfs["pyroptosis_shuffled_validation_df"] = pyroptosis_shuffled_validation_df
dict_of_dfs[
    "pyroptosis_shuffled_treatment_holdout_df"
] = pyroptosis_shuffled_treatment_holdout_df
dict_of_dfs["pyroptosis_shuffled_holdout_df"] = pyroptosis_shuffled_holdout_df

dict_of_dfs["pyroptosis_unshuffled_train_df"] = pyroptosis_unshuffled_train_df
dict_of_dfs["pyroptosis_unshuffled_test_df"] = pyroptosis_unshuffled_test_df
dict_of_dfs["pyroptosis_unshuffled_validation_df"] = pyroptosis_unshuffled_validation_df
dict_of_dfs[
    "pyroptosis_unshuffled_treatment_holdout_df"
] = pyroptosis_unshuffled_treatment_holdout_df
dict_of_dfs["pyroptosis_unshuffled_holdout_df"] = pyroptosis_unshuffled_holdout_df

dict_of_dfs["apoptosis_shuffled_train_df"] = apoptosis_shuffled_train_df
dict_of_dfs["apoptosis_shuffled_test_df"] = apoptosis_shuffled_test_df
dict_of_dfs["apoptosis_shuffled_validation_df"] = apoptosis_shuffled_validation_df
dict_of_dfs[
    "apoptosis_shuffled_treatment_holdout_df"
] = apoptosis_shuffled_treatment_holdout_df
dict_of_dfs["apoptosis_shuffled_holdout_df"] = apoptosis_shuffled_holdout_df

dict_of_dfs["apoptosis_unshuffled_train_df"] = apoptosis_unshuffled_train_df
dict_of_dfs["apoptosis_unshuffled_test_df"] = apoptosis_unshuffled_test_df
dict_of_dfs["apoptosis_unshuffled_validation_df"] = apoptosis_unshuffled_validation_df
dict_of_dfs[
    "apoptosis_unshuffled_treatment_holdout_df"
] = apoptosis_unshuffled_treatment_holdout_df
dict_of_dfs["apoptosis_unshuffled_holdout_df"] = apoptosis_unshuffled_holdout_df

dict_of_dfs["control_shuffled_train_df"] = control_shuffled_train_df
dict_of_dfs["control_shuffled_test_df"] = control_shuffled_test_df
dict_of_dfs["control_shuffled_validation_df"] = control_shuffled_validation_df
dict_of_dfs[
    "control_shuffled_treatment_holdout_df"
] = control_shuffled_treatment_holdout_df
dict_of_dfs["control_shuffled_holdout_df"] = control_shuffled_holdout_df

dict_of_dfs["control_unshuffled_train_df"] = control_unshuffled_train_df
dict_of_dfs["control_unshuffled_test_df"] = control_unshuffled_test_df
dict_of_dfs["control_unshuffled_validation_df"] = control_unshuffled_validation_df
dict_of_dfs[
    "control_unshuffled_treatment_holdout_df"
] = control_unshuffled_treatment_holdout_df
dict_of_dfs["control_unshuffled_holdout_df"] = control_unshuffled_holdout_df

In [9]:
dict_of_dfs
dict_of_dfs["control_unshuffled_holdout_df"]

,true_label,predicted_label,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,...,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,labels,data_split,shuffle,correct
1044903,1,1,SH-SY5Y,F14,4528,1,6,DMSO,0.025,%,...,1804.679571,40.531621,1847.0,83.0,1751.0,8.0,healthy,holdout,False,True
1044904,1,1,SH-SY5Y,F14,4528,1,6,DMSO,0.025,%,...,1635.391284,62.055169,1687.0,139.0,1608.0,22.0,healthy,holdout,False,True
1044905,1,1,SH-SY5Y,F14,4528,1,6,DMSO,0.025,%,...,729.948915,71.122966,777.0,137.0,673.0,34.0,healthy,holdout,False,True
1044906,1,1,SH-SY5Y,F14,4528,1,6,DMSO,0.025,%,...,1720.766706,73.073857,1767.0,161.0,1673.0,22.0,healthy,holdout,False,True
1044907,1,1,SH-SY5Y,F14,4528,1,6,DMSO,0.025,%,...,821.125208,96.633527,875.0,139.0,776.0,52.0,healthy,holdout,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195799,1,1,SH-SY5Y,K23,3367,16,6,Z-VAD-FMK,100.000,µM,...,1113.145483,759.886817,1245.0,846.0,1051.0,670.0,healthy,holdout,False,True
1195800,1,1,SH-SY5Y,K23,3367,16,6,Z-VAD-FMK,100.000,µM,...,1294.181266,940.952405,1348.0,1001.0,1223.0,914.0,healthy,holdout,False,True
1195801,1,1,SH-SY5Y,K23,3367,16,6,Z-VAD-FMK,100.000,µM,...,317.480482,1097.865672,434.0,1189.0,295.0,1070.0,healthy,holdout,False,True
1195802,1,1,SH-SY5Y,K23,3367,12,6,Z-VAD-FMK,100.000,µM,...,938.990239,477.994332,1005.0,525.0,909.0,436.0,healthy,holdout,False,True


In [10]:
# function that selects a random image from the dataframe
def random_cell_select(df: pd.DataFrame) -> pd.DataFrame:
    """
    Selects a random cell from the dataframe

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe containing the cell features

    Returns
    -------
    pd.DataFrame
        The return dataframe with the random cell selected
    """

    # select a random cell
    random_cell = df.sample(n=1, random_state=0)
    return random_cell

In [11]:
control_unshuffled_holdout_cell = random_cell_select(
    dict_of_dfs["control_unshuffled_holdout_df"]
)
control_unshuffled_holdout_cell
# define parts of the image from the df

,true_label,predicted_label,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_Site,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,...,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,labels,data_split,shuffle,correct
1047884,1,1,SH-SY5Y,F14,4528,6,6,DMSO,0.025,%,...,519.909735,707.377844,581.0,759.0,484.0,653.0,healthy,holdout,False,True


In [12]:
image_id = control_unshuffled_holdout_cell["Metadata_ImageNumber"].values[0]
fov_id = control_unshuffled_holdout_cell["Metadata_Site"].astype(str).values[0]
cell_id = control_unshuffled_holdout_cell["Metadata_Cells_Number_Object_Number"].values[
    0
]
well_id = control_unshuffled_holdout_cell["Metadata_Well"].values[0]
row_id = well_id[0]
column_id = well_id[1:]
center_x = control_unshuffled_holdout_cell["Metadata_Nuclei_Location_Center_X"].values[
    0
]
center_y = control_unshuffled_holdout_cell["Metadata_Nuclei_Location_Center_Y"].values[
    0
]
# median_row['Cytoplasm_']
# 'Cytoplasm_AreaShape_BoundingBoxMaximum_X', 'Cytoplasm_AreaShape_BoundingBoxMaximum_Y', 'Cytoplasm_AreaShape_BoundingBoxMinimum_X', 'Cytoplasm_AreaShape_BoundingBoxMinimum_Y'
max_x_box = (
    control_unshuffled_holdout_cell["Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X"]
    .values[0]
    .astype(int)
)
max_y_box = (
    control_unshuffled_holdout_cell["Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y"]
    .values[0]
    .astype(int)
)
min_x_box = (
    control_unshuffled_holdout_cell["Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X"]
    .values[0]
    .astype(int)
)
min_y_box = (
    control_unshuffled_holdout_cell["Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y"]
    .values[0]
    .astype(int)
)
print(
    f"Image ID: {image_id}",
    f"Cell ID: {cell_id}",
    f"Well ID: {well_id}",
    f"row_id: {row_id}",
    f"fov_id: {fov_id}",
    f"column id: {column_id}",
    f"Center X: {center_x}",
    f"Center Y: {center_y}",
    f"Max X: {max_x_box}",
    f"Max Y: {max_y_box}",
    f"Min X: {min_x_box}",
    f"Min Y: {min_y_box}",
    sep="\n",
)

Image ID: 726
Cell ID: 94
Well ID: F14
row_id: F
fov_id: 6
column id: 14
Center X: 519.9097351734428
Center Y: 707.3778440880269
Max X: 581
Max Y: 759
Min X: 484
Min Y: 653


In [13]:
well_dict = {
    "A": "01",
    "B": "02",
    "C": "03",
    "D": "04",
    "E": "05",
    "F": "06",
    "G": "07",
    "H": "08",
    "I": "09",
    "J": "10",
    "K": "11",
    "L": "12",
    "M": "13",
    "N": "14",
    "O": "15",
    "P": "16",
}
column_dict = {
    "1": "01",
    "2": "02",
    "3": "03",
    "4": "04",
    "5": "05",
    "6": "06",
    "7": "07",
    "8": "08",
    "9": "09",
    "10": "10",
    "11": "11",
    "12": "12",
    "13": "13",
    "14": "14",
    "15": "15",
    "16": "16",
    "17": "17",
    "18": "18",
    "19": "19",
    "20": "20",
    "21": "21",
    "22": "22",
    "23": "23",
    "24": "24",
}
fov_dict = {
    "1": "01",
    "2": "02",
    "3": "03",
    "4": "04",
    "5": "05",
    "6": "06",
    "7": "07",
    "8": "08",
    "9": "09",
    "10": "10",
    "11": "11",
    "12": "12",
    "13": "13",
    "14": "14",
    "15": "15",
    "16": "16",
}

In [14]:
image_basename_1 = "p04-ch1sk1fk1fl1_IC.tiff"
image_basename_2 = "p04-ch2sk1fk1fl1_IC.tiff"
image_basename_3 = "p04-ch3sk1fk1fl1_IC.tiff"
image_basename_4 = "p04-ch4sk1fk1fl1_IC.tiff"
image_basename_5 = "p04-ch5sk1fk1fl1_IC.tiff"

In [15]:
print(row_id, column_id, image_id, fov_id)
print(f"r{well_dict[row_id]}c{column_dict[column_id]}f{fov_dict[fov_id]}")

F 14 726 6
r06c14f06


In [16]:
image_name1 = f"r{well_dict[row_id]}c{column_dict[column_id]}f{fov_dict[fov_id]}{image_basename_1}"
image_path1 = image_dir_path.joinpath(image_name1)
print(image_name1, "\n", image_path1)

image_name2 = f"r{well_dict[row_id]}c{column_dict[column_id]}f{fov_dict[fov_id]}{image_basename_2}"
image_path2 = image_dir_path.joinpath(image_name2)
print(image_name2, "\n", image_path2)

image_name3 = f"r{well_dict[row_id]}c{column_dict[column_id]}f{fov_dict[fov_id]}{image_basename_3}"
image_path3 = image_dir_path.joinpath(image_name3)
print(image_name3, "\n", image_path3)

image_name4 = f"r{well_dict[row_id]}c{column_dict[column_id]}f{fov_dict[fov_id]}{image_basename_4}"
image_path4 = image_dir_path.joinpath(image_name4)
print(image_name4, "\n", image_path4)

image_name5 = f"r{well_dict[row_id]}c{column_dict[column_id]}f{fov_dict[fov_id]}{image_basename_5}"
image_path5 = image_dir_path.joinpath(image_name5)
print(image_name5, "\n", image_path5)

r06c14f06p04-ch1sk1fk1fl1_IC.tiff 
 /media/lippincm/18T/interstellar_data/70117_20230210MM1_Gasdermin514_CP_BC430856__2023-03-22T15_42_38-Measurement1/2.IC/r06c14f06p04-ch1sk1fk1fl1_IC.tiff
r06c14f06p04-ch2sk1fk1fl1_IC.tiff 
 /media/lippincm/18T/interstellar_data/70117_20230210MM1_Gasdermin514_CP_BC430856__2023-03-22T15_42_38-Measurement1/2.IC/r06c14f06p04-ch2sk1fk1fl1_IC.tiff
r06c14f06p04-ch3sk1fk1fl1_IC.tiff 
 /media/lippincm/18T/interstellar_data/70117_20230210MM1_Gasdermin514_CP_BC430856__2023-03-22T15_42_38-Measurement1/2.IC/r06c14f06p04-ch3sk1fk1fl1_IC.tiff
r06c14f06p04-ch4sk1fk1fl1_IC.tiff 
 /media/lippincm/18T/interstellar_data/70117_20230210MM1_Gasdermin514_CP_BC430856__2023-03-22T15_42_38-Measurement1/2.IC/r06c14f06p04-ch4sk1fk1fl1_IC.tiff
r06c14f06p04-ch5sk1fk1fl1_IC.tiff 
 /media/lippincm/18T/interstellar_data/70117_20230210MM1_Gasdermin514_CP_BC430856__2023-03-22T15_42_38-Measurement1/2.IC/r06c14f06p04-ch5sk1fk1fl1_IC.tiff


In [17]:
# crop all 5 channels of the image
im1 = cv2.imread(image_path1.as_posix(), cv2.IMREAD_GRAYSCALE)
# im1 = cv2.convertScaleAbs(im1, alpha=alpha, beta=beta)
im_crop1 = im1[min_y_box:max_y_box, min_x_box:max_x_box]

im2 = cv2.imread(image_path2.as_posix(), cv2.IMREAD_GRAYSCALE)
# im2 = cv2.convertScaleAbs(im2, alpha=alpha, beta=beta)
im_crop2 = im2[min_y_box:max_y_box, min_x_box:max_x_box]

im3 = cv2.imread(image_path3.as_posix(), cv2.IMREAD_GRAYSCALE)
# im3 = cv2.convertScaleAbs(im3, alpha=alpha, beta=beta)
im_crop3 = im3[min_y_box:max_y_box, min_x_box:max_x_box]

im4 = cv2.imread(image_path4.as_posix(), cv2.IMREAD_GRAYSCALE)
# im4 = cv2.convertScaleAbs(im4, alpha=alpha, beta=beta)
im_crop4 = im4[min_y_box:max_y_box, min_x_box:max_x_box]

im5 = cv2.imread(image_path5.as_posix(), cv2.IMREAD_GRAYSCALE)
# im5 = cv2.convertScaleAbs(im5, alpha=alpha, beta=beta)
im_crop5 = im5[min_y_box:max_y_box, min_x_box:max_x_box]

### channels
* Channel 1: DAPI
* Channel 2: ER
* Channel 3: GasderminD
* Channel 4: AGP (Actin, Golgi, and Plasma membrane)
* Channel 5: Mitochondria


In [18]:
# pick three channels to stack
# nuclei = blue
# Gasdermin = green
# Actin = red

blue_channel_stack = np.stack(im1, axis=-1)
green_channel_stack = np.stack(im3, axis=-1)
red_channel_stack = np.stack(im4, axis=-1)

channel1 = "im1"
channel2 = "im3"
channel3 = "im4"

# Scale the pixel values to fit within the 16-bit range (0-65535)
blue_channel = (blue_channel_stack / np.max(blue_channel_stack) * 65535).astype(
    np.uint16
)
green_channel = (green_channel_stack / np.max(green_channel_stack) * 65535).astype(
    np.uint16
)
red_channel = (red_channel_stack / np.max(red_channel_stack) * 65535).astype(np.uint16)

In [19]:
composite_image = cv2.merge((blue_channel, green_channel, red_channel)).astype(
    np.uint16
)
composite_image.shape
composite_image = cv2.cvtColor(composite_image, cv2.COLOR_BGR2RGB)

In [20]:
# transformations of the image to fix the orientation post pixel scaling
# flip the image vertically
composite_image = cv2.flip(composite_image, 0)
# rotate the image 90 degrees clockwise
composite_image = cv2.rotate(composite_image, cv2.ROTATE_90_CLOCKWISE)

In [21]:
# cv2.imshow("Composite", composite_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [22]:
# crop the composite image
im_crop = composite_image[min_y_box:max_y_box, min_x_box:max_x_box]
# cv2.imshow("Composite", im_crop)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [23]:
# image_out_dir_path updated to include the feature name
image_out_dir_path = pathlib.Path(f"{image_out_dir_path}/{feature}")
image_out_dir_path.mkdir(parents=True, exist_ok=True)
# write images
tf.imwrite(
    pathlib.Path(
        f"{image_out_dir_path}/{channel1}_{channel2}_{channel3}_composite_image.tiff"
    ),
    composite_image,
    compression=None,
)
tf.imwrite(
    pathlib.Path(
        f"{image_out_dir_path}/{channel1}_{channel2}_{channel3}_composite_image_crop.tiff"
    ),
    im_crop,
    compression=None,
)